# Cuery usage example

In [1]:
import importlib
from pathlib import Path
from pprint import pprint

import instructor
import openai
import pandas as pd
from rich.pretty import pprint

from cuery import utils
from cuery.work import DirceJobs, DirceTasks

GDRIVE = Path("~/Google Drive/Shared drives/Solutions").expanduser()

# Jobs

In [2]:
DATA_DIR = GDRIVE / "Research/future_of_work/inputs/ine_dirce_aggregated_by_activity.csv"
data = pd.read_csv(DATA_DIR)
data.columns = [c.replace(" ", "_") for c in data.columns]
display(data)

n_examples = 5
context_df = data.iloc[:5]
context_dict = {col: context_df[col].tolist() for col in context_df.columns}
context_records = context_df.to_dict(orient="records")

,Division,Actividad_principal,Estimated_Employees_2024,Median_YoY_Growth_pct
0,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4
1,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2
2,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",303430,0.5
3,45 Venta y reparación de vehículos de motor y ...,451 Venta de vehículos de motor,102172,-0.5
4,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9
5,46 Comercio al por mayor e intermediarios del ...,464 Comercio al por mayor de artículos de uso ...,228324,-2.0
6,"47 Comercio al por menor, excepto de vehículos...",471 Comercio al por menor en establecimientos ...,233708,1.3
7,"47 Comercio al por menor, excepto de vehículos...",477 Comercio al por menor de otros artículos e...,242341,-1.3
8,49 Transporte terrestre y por tubería,493 Otro transporte terrestre de pasajeros,142413,0.5
9,49 Transporte terrestre y por tubería,494 Transporte de mercancías por carretera y s...,273002,-1.5


In [3]:
jobs = await DirceJobs(context_df, model="gpt-4.1-mini", n_concurrent=2)
jobs

Gathering responses: 100%|██████████| 5/5 [00:07<00:00,  1.60s/it]


[Jobs(jobs=[Job(name='Quality Control Analyst', description='Inspecting and analyzing meat products for quality assurance.', automation_potential=7, reason='Image recognition and data analysis can automate inspections.'), Job(name='Production Planner', description='Scheduling production runs and managing inventory for meat processing.', automation_potential=8, reason='Demand forecasting and scheduling algorithms reduce manual planning.'), Job(name='Compliance Documentation Specialist', description='Preparing and managing regulatory and safety documentation.', automation_potential=9, reason='Document generation and compliance tracking can be automated by AI.')]),
 Jobs(jobs=[Job(name='Production Planner', description='Organizes manufacturing schedules and resource allocation to meet production goals efficiently.', automation_potential=7, reason='Routine scheduling and inventory management can be optimized by AI software.'), Job(name='Quality Control Analyst', description='Inspects and a

In [4]:
jobs_df = DirceJobs.explode_responses(jobs, context_df)
jobs_df = jobs_df.rename(
    columns={
        "name": "job",
        "description": "job_description",
        "automation_potential": "job_automation_potential",
        "reason": "job_automation_reason",
    }
)
jobs_df

,Division,Actividad_principal,Estimated_Employees_2024,Median_YoY_Growth_pct,job,job_description,job_automation_potential,job_automation_reason
0,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Quality Control Analyst,Inspecting and analyzing meat products for qua...,7,Image recognition and data analysis can automa...
1,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Production Planner,Scheduling production runs and managing invent...,8,Demand forecasting and scheduling algorithms r...
2,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Compliance Documentation Specialist,Preparing and managing regulatory and safety d...,9,Document generation and compliance tracking ca...
3,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Production Planner,Organizes manufacturing schedules and resource...,7,Routine scheduling and inventory management ca...
4,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Quality Control Analyst,Inspects and analyzes production samples to en...,6,AI can analyze patterns in quality data and fl...
5,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2,Supply Chain Coordinator,Coordinates logistics of materials and compone...,7,AI excels at optimizing supply chains with pre...
6,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",303430,0.5,Administrative Assistant,"Handles scheduling, paperwork, and record keep...",8,Routine document handling and scheduling can b...
7,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",303430,0.5,Estimador de Costos,Prepares cost estimates for electrical and plu...,7,AI can analyze past data for accurate cost est...
8,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",303430,0.5,Technical Drafter,Creates technical drawings and plans for elect...,6,AI can assist in drafting plans but human over...
9,45 Venta y reparación de vehículos de motor y ...,451 Venta de vehículos de motor,102172,-0.5,Car Salesperson,Handles customer interactions and sales proces...,6,"AI can automate customer queries, pricing, and..."


## Tasks

In [5]:
tasks = await DirceTasks(jobs_df, model="gpt-4.1-mini", n_concurrent=2)
tasks

Gathering responses: 100%|██████████| 15/15 [00:47<00:00,  3.14s/it]

Encountered: 1 response parsing errors!


[JobTasks(tasks=[JobTask(name='Data entry and report generation', description='Automate the input of quality control data and generate standardized reports.', automation_potential=9, intelligence=3, sexyness=4, scalability=8, data_needs='Quality control results data, report templates and formats', products=['Microsoft Power Automate', 'Tableau', 'Zoho Analytics']), JobTask(name='Sample tracking and logging', description='Automate the tracking and logging of samples through the processing stages.', automation_potential=8, intelligence=4, sexyness=5, scalability=7, data_needs='Sample ID data, processing stage information, timestamps', products=['LabWare LIMS', 'Qiagen Digital Insights', 'Sample Manager']), JobTask(name='Non-compliance and defect detection', description='Automatically analyze quality data to detect non-compliance and defects based on set thresholds.', automation_potential=8, intelligence=7, sexyness=6, scalability=8, data_needs='Quality measurements, defect records, compl

In [6]:
tasks_df = DirceTasks.explode_responses(tasks, jobs_df)
tasks_df = tasks_df.rename(columns={"name": "task", "description": "task_description"})
tasks_df

,Division,Actividad_principal,Estimated_Employees_2024,Median_YoY_Growth_pct,job,job_description,job_automation_potential,job_automation_reason,task,task_description,automation_potential,intelligence,sexyness,scalability,data_needs,products
0,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Quality Control Analyst,Inspecting and analyzing meat products for qua...,7,Image recognition and data analysis can automa...,Data entry and report generation,Automate the input of quality control data and...,9,3,4,8,"Quality control results data, report templates...","[Microsoft Power Automate, Tableau, Zoho Analy..."
1,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Quality Control Analyst,Inspecting and analyzing meat products for qua...,7,Image recognition and data analysis can automa...,Sample tracking and logging,Automate the tracking and logging of samples t...,8,4,5,7,"Sample ID data, processing stage information, ...","[LabWare LIMS, Qiagen Digital Insights, Sample..."
2,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Quality Control Analyst,Inspecting and analyzing meat products for qua...,7,Image recognition and data analysis can automa...,Non-compliance and defect detection,Automatically analyze quality data to detect n...,8,7,6,8,"Quality measurements, defect records, complian...","[Minitab, JMP Statistical Software, SPSS]"
3,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Quality Control Analyst,Inspecting and analyzing meat products for qua...,7,Image recognition and data analysis can automa...,Document control and management,"Automate the organization, version control and...",7,5,4,7,"Document files, version history, user access logs","[DocuWare, M-Files, SharePoint]"
4,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4,Quality Control Analyst,Inspecting and analyzing meat products for qua...,7,Image recognition and data analysis can automa...,Trend analysis and forecasting,Use AI to analyze quality trends over time and...,7,8,7,7,"Historical quality control data, process param...","[Tableau, Power BI, SAS Visual Analytics]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9,Customer Service Representative,Handles inquiries and complaints related to wh...,7,AI chatbots can handle routine customer querie...,Order Processing,Handling and entering customer orders into the...,8,6,5,8,"Order details, customer data, product catalogs","[Salesforce Order Management, Zoho Inventory]"
72,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9,Customer Service Representative,Handles inquiries and complaints related to wh...,7,AI chatbots can handle routine customer querie...,Customer Inquiry Response,Responding to customer questions about product...,7,7,6,7,"Customer inquiries, product information, polic...","[Zendesk, Freshdesk]"
73,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9,Customer Service Representative,Handles inquiries and complaints related to wh...,7,AI chatbots can handle routine customer querie...,Complaint Management,Logging and tracking customer complaints for r...,6,7,4,6,"Complaint details, customer records, resolutio...","[Zoho Desk, HubSpot Service Hub]"
74,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9,Customer Service Representative,Handles inquiries and complaints related to wh...,7,AI chatbots can handle routine customer querie...,Invoice Generation,Creating and sending invoices to customers aft...,8,5,5,8,"Order and payment details, customer billing info","[QuickBooks, Xero]"


In [9]:
pprint(DirceJobs.prompt)
pprint(DirceJobs.response.model_json_schema())

Prompt(
│   messages=[
│   │   Message(
│   │   │   role='system',
│   │   │   content="You're an analyst at the Spanish 'Instituo Nacional de Estadística' (INE) analyzing\ndata from its 'Directorio Central de Empresas' (DIRCE). Your objective is to analyze\ngroups of companies, identified by a sector ('Division') and a corresponding main activity\n('Actividad principal') in order to identify jobs within those companies that are likely to\nbe automatable by AI. Both 'Division' and 'Actividad_principal' are provided in Spanish and may\ninclude numeric IDs that you can ignore if you don't understand them. Always respond in English.\nOnly consider jobs that are computer- or paper-based and can be automated by AI using software\n(don't include jobs automatable by robots or other physical means)."
│   │   ),
│   │   Message(
│   │   │   role='user',
│   │   │   content='Please analyze the following jobs sector and identify jobs that are automatable by AI software.\nDivision: {{Division}}\nActividad principal: {{Actividad_principal}}'
│   │   )
│   ],
│   required=['Division', 'Actividad_principal']
)

{
│   '$defs': {
│   │   'Job': {
│   │   │   'properties': {
│   │   │   │   'name': {
│   │   │   │   │   'description': 'Name of the job role (job title, less than 50 characters)',
│   │   │   │   │   'maxLength': 50,
│   │   │   │   │   'minLength': 5,
│   │   │   │   │   'title': 'Name',
│   │   │   │   │   'type': 'string'
│   │   │   │   },
│   │   │   │   'description': {
│   │   │   │   │   'description': 'A short description of the job role (less than 200 characters)',
│   │   │   │   │   'maxLength': 200,
│   │   │   │   │   'minLength': 20,
│   │   │   │   │   'title': 'Description',
│   │   │   │   │   'type': 'string'
│   │   │   │   },
│   │   │   │   'automation_potential': {
│   │   │   │   │   'description': "A score from 1 to 10 indicating the job's potential for automation",
│   │   │   │   │   'maximum': 10,
│   │   │   │   │   'minimum': 0,
│   │   │   │   │   'title': 'Automation Potential',
│   │   │   │   │   'type': 'integer'
│   │   │   │   },
│   │   │   │   'reason': {
│   │   │   │   │   'description': 'A short explanation of no more than 10 words, of why the job is likely to be automatable with the given potential score.',
│   │   │   │   │   'maxLength': 300,
│   │   │   │   │   'minLength': 30,
│   │   │   │   │   'title': 'Reason',
│   │   │   │   │   'type': 'string'
│   │   │   │   }
│   │   │   },
│   │   │   'required': ['name', 'description', 'automation_potential', 'reason'],
│   │   │   'title': 'Job',
│   │   │   'type': 'object'
│   │   }
│   },
│   'properties': {
│   │   'jobs': {
│   │   │   'description': 'A list of jobs with their AI automation potential and reasons for that potential',
│   │   │   'items': {'$ref': '#/$defs/Job'},
│   │   │   'minItems': 3,
│   │   │   'title': 'Jobs',
│   │   │   'type': 'array'
│   │   }
│   },
│   'required': ['jobs'],
│   'title': 'Jobs',
│   'type': 'object'
}